## Attention Is All You Need

<img src='https://github.com/bentrevett/pytorch-seq2seq/raw/e8209a7b0207cde55871be352819cac3dd5c05ce/assets/transformer1.png'>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import os
import random
import math
import time

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [6]:
seed_everything(1234)

BATCH_SIZE = 128

device = "cuda" if torch.cuda.is_available() else "cpu" 

In [7]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [8]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [9]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [10]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

In [11]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [12]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

## ENCODER

<img src='https://github.com/bentrevett/pytorch-seq2seq/raw/e8209a7b0207cde55871be352819cac3dd5c05ce/assets/transformer-encoder.png'>

이전의 기존 Attention모델에서처럼, Source문장을 하나의 Context Vector $z$로 나타내지 않습니다.

대신에, Soruce문장 길이에 맞는 Sequence of Context Vectors $Z = (z_1, ... , z_n)$를 만듭니다.

* *example: len(source):5이면, $Z = (z_1, z_2, z_3, z_4, z_5)$*

왜 sequence of hidden states라 부르지 않고, sequence of context vectors라 부르는 이유는 무엇일까?

$t$시점의 hidden state는 $x_t$의 token과 그 전까지의 $h_{t-1}$을 보지만,

context vector는 source문장 내의 모든 위치의 token을 취합니다.

첫번쨰로, token들이 embedding layer를 통과합니다.

다음에는, 모델이 input sequence안에서의 token들의 순서를 모르기 때문에, 

positional embedding layer라 불리는 두 번쨰 embedding layer를 통과시킵니다.

첫번쨰 embedding layer에서의 input은 token 그 자체가 아니라, sequence안의 token 위치입니다.

첫 token, $<SOS>$ token, posion 0부터 시작합니다.
    
position embedding은 vocab의 사이즈를 100으로 하였을떄, 모델은 문장을 100개의 token으로 인식합니다. 

원래의 Attention is All You Need 논문에서 나온 Transfomer는 positional embedding을 학습하지 않습니다.

대신에 고정된 길이의 embedding을 사용하였습니다.

하지만 BERT처럼  최신의 Transformer구조는 positional embedding을 사용합니다.

따라서 이 chapter에서도 positional embedding을 사용합니다.

다음에는 token과 postional embedding을 elementwise 방식으로 더하여 하나의 vector를 얻습니다.

이 vector에는 위치와 token에 대한 정보를 포함합니다.

합치기 전에는 scaling factor which is $\sqrt{d_{model}}$, where $d_{model}$ is the hidden dimension size, hid_dim를 token embedding과 곱해주어야 합니다.

이것은 임베딩에서의 분산을 감소시키는 것으로 추정되며, scaling factor가 없으면 모델을 안정적으로 훈련시키기가 어렵습니다.

결합된 embeddings들은 Dropout을 적용합니다.

결합된 embeddings는 $N$개의 encoder layer를 통과 후, $Z$를 얻게되며, 다시 decoder에 의해 사용됩니다.

source mask, src_mask는 간단히 source 문장에서의 같은 shape를 가지면서, $<pad>$ token일 경우 0, 아닐 경우 1을 가지게 됩니다.

이것이 encoder layer에 사용된 multi-head attention mechanism의 원리이며,soruce 문장 내에서 attention을 적용합니다.

모델은 $<pad>$tokden에 attention하지 않고, 쓸모없는 정보를 포함하지 않습니다.
 

In [4]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device, max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, n_heads, pf_dim, dropout, device) for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [40]:
batch = next(iter(train_iterator))
test_src = batch.src
test_src = test_src.view(128,-1)

In [52]:
test_src

tensor([[2, 2, 2,  ..., 2, 2, 2],
        [2, 2, 2,  ..., 2, 2, 2],
        [2, 2, 2,  ..., 2, 2, 2],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')

In [42]:
test_src.size()

torch.Size([128, 36])

In [44]:
batch_size = test_src.shape[0]
src_len = test_src.shape[1]

In [50]:
pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1)

In [51]:
pos

tensor([[ 0,  1,  2,  ..., 33, 34, 35],
        [ 0,  1,  2,  ..., 33, 34, 35],
        [ 0,  1,  2,  ..., 33, 34, 35],
        ...,
        [ 0,  1,  2,  ..., 33, 34, 35],
        [ 0,  1,  2,  ..., 33, 34, 35],
        [ 0,  1,  2,  ..., 33, 34, 35]])

### Encoder Layer

<img src='https://yjucho1.github.io/assets/img/2018-10-13/transformer-encoder.png'>

The encoder layers are where all of the "meat" of the encoder is contained. 

encoder layers들은 모두 multi head attention layer가 포함되어 있습니다.

>We first pass the source sentence and its mask into the multi-head attention layer, then perform dropout on it, apply a residual >connection and pass it through a **Layer Normalization layer**.
>
>We then pass it through a position-wise feedforward layer and then, again, apply dropout, a residual connection and then layer >normalization to get the output of this layer which is fed into the next layer.
>
>The parameters are not shared between layers.
>
mutli head attention layer는 encoder layer에 의해, source 문장을 attend 하는데 사용됩니다.

즉, 다른 sequence 대신 자체적으로 atteintion를 계산하고 적용하므로 self attention이라고 합니다.

이를 통해 Transformer와 같이 더 많은 계층의 신경망을 쉽게 학습 할 수 있습니다.

In [53]:
class EncoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim,  dropout, device):
        super().__init__()
        self.layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, src len]
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

### Mutli Head Attention Layer

<img src='https://github.com/bentrevett/pytorch-seq2seq/raw/e8209a7b0207cde55871be352819cac3dd5c05ce/assets/transformer-attention.png'>

Multi-Head Attention의 역할은 1문장을 여러 head로 Self-Attention 시킴에 있습니다.

$$ \text{Attention}(Q, K, V) = \text{Softmax} \big( \frac{QK^T}{\sqrt{d_k}} \big)V $$

* Query(Q) : 영향을 받는 단어 A를 나타내는 변수입니다.
* Key(K) : 영향을 주는 단어 B를 나타내는 변수입니다.
* Value(V) : 그 영향에 대한 가중치를 나타냅니다.


"Je suis étudiant" 라는 문장의 임베딩 벡터가 512차원이라면 8개 head로 나눠 64개의 벡터를 한 Scaled Dot Attention이 맡아 처리하는 것입니다. 

In [55]:
# self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
# _src, _ = self.self_attention(src, src, src, src_mask)

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim 
        self.n_heads = n_heads 
        self.head_dim = hid_dim // n_heads 
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

### Position-wise Feedforward Layer

각 head가 만들어낸 Self-Attention을 치우치지 않게 균등하게 섞는 역할을 합니다. 

In [78]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x